In [50]:
import redcap
import csv
from datetime import datetime, timedelta
import pandas as pd

# REDCap API endpoint and token
data = {
    'token': 'DE12CA04415CF93BCD68321EBF5040FD',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])
data = {
    'token': 'D6807C091B1FC14F57CC9B2972D0B398',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
indigo=redcap.Project('https://ruff.mrc.gm:8443/redcap/api/',data['token'])

In [51]:
indigo_data=indigo.export_records(forms=['consent','delivery', 'ultrasound_scan'])
indigo_data=pd.DataFrame(indigo_data)
indigo_data_eden=indigo_data[
    (indigo_data['con_participant_eden_num_q11']!='')
]


#indigo_data_del=indigo_data[
    #(indigo_data['delivery_complete']=='2')|
    #(indigo_data['delivery_complete']=='1')
#]

#join_edn_del=pd.merge(indigo_data_del,indigo_data_eden,on='con_participantid_q1',how='inner')
#join_edn_del=join_edn_del[['con_participantid_q1','con_participant_eden_num_q11_y','delivery_complete_x']]
#join_edn_del=join_edn_del.rename(columns={'con_participant_eden_num_q11_y':'participant_id'})

In [52]:
# Convert columns to appropriate types
indigo_data_eden=indigo_data_eden[['con_participantid_q1','con_participant_eden_num_q11','ultsc_date','ultrascan_exp_date','ga_weeks', 'ga_days']]
indigo_data_eden=indigo_data_eden.rename(columns={'con_participant_eden_num_q11':'participant_id'})
indigo_data_eden['ga_weeks'] = pd.to_numeric(indigo_data_eden['ga_weeks'])
indigo_data_eden['ga_days'] = pd.to_numeric(indigo_data_eden['ga_days'])
indigo_data_eden['converted_weeks_todays'] = indigo_data_eden['ga_weeks'] * 7 + indigo_data_eden['ga_days']
indigo_data_eden['ultsc_date'] = pd.to_datetime(indigo_data_eden['ultsc_date'])
indigo_data_eden['ultrascan_exp_date'] = pd.to_datetime(indigo_data_eden['ultrascan_exp_date'])
# Filter out rows with empty ultrascan_exp_date
indigo_data_eden = indigo_data_eden[indigo_data_eden['ultrascan_exp_date'].notna()]
# Calculate conception date
indigo_data_eden['conception_date'] = indigo_data_eden['ultsc_date'] - pd.to_timedelta(indigo_data_eden['converted_weeks_todays'], unit='D')


In [53]:
df_ultra=df.export_records(forms=['ultrasound_scan'])
df_ultra=pd.DataFrame(df_ultra)
df_ultra=df_ultra[
    (df_ultra['ultrasound_scan_complete']=='2')|
    (df_ultra['ultrasound_scan_complete']=='1')&
    (df_ultra['conception_date']!='')
].drop_duplicates('participant_id')
df_ultra=df_ultra[['participant_id','ultrascan_date','conception_date','convert_week_today']]
df_ultra['ultrascan_date']=pd.to_datetime(df_ultra['ultrascan_date'])
df_ultra['convert_week_today']=pd.to_numeric(df_ultra['convert_week_today'])


In [54]:
df_ultra_margh=pd.merge(df_ultra,indigo_data_eden,on='participant_id',how='left')
df_ultra_margh.to_csv('df_ultra_margh.csv',index=False)

In [55]:
df_del=df.export_records(forms=['delivery'])
df_del=pd.DataFrame(df_del)
df_del=df_del[
    (df_del['delivery_complete']=='2')|
    (df_del['delivery_complete']=='1')
]
#df_del=df_del[['participant_id','del_ddate_q7','del_birthweight_avai_q11','del_birthweight_q11a','del_sex_q10','delivery_complete']]
#df_del.to_csv('df_del.csv')

In [56]:
df_cordcollection=df.export_records(forms=['cordcollection'])
df_cordcollection=pd.DataFrame(df_cordcollection)
df_cordcollection=df_cordcollection[
    (df_cordcollection['cordcollection_complete']=='2')|
    (df_cordcollection['cordcollection_complete']=='1')
]
df_cordcollection=df_cordcollection[['participant_id','cordcollection_complete']]

df_withdraw=df.export_records(forms=['withdrawal'])
df_withdraw=pd.DataFrame(df_withdraw)
df_withdraw=df_withdraw[
    (df_withdraw['withdrawal_complete']=='2')
]
#del_not_withdrawl=df_del[~df_del['participant_id'].isin(df_withdraw['participant_id'])]
#del_not_withdrawl.to_csv('del_not_withdrawl.csv')

df_del_not_withrawl_mergh=pd.merge(df_cordcollection,df_del,on='participant_id',how='left')
df_del_not_withrawl_mergh_1=pd.merge(df_del_not_withrawl_mergh,df_withdraw,on='participant_id',how='left')
df_del_not_withrawl_mergh_1.to_csv('df_del_not_withrawl_mergh.csv',index=False)

In [57]:
df_sample_collection=df.export_records(forms=['preganancy_stool_collection','maternal_blood_sample_collection','preganancy_urine_collection'])
df_sample_collection=pd.DataFrame(df_sample_collection)

df_sample_collection=df_sample_collection[
    (df_sample_collection['urinecollected']=='1')|
    (df_sample_collection['mstool_time']=='1')|
    (df_sample_collection['matbld_visitdate_q1_e327e1']!='')   
]
 
df_stool=df_sample_collection[
 (df_sample_collection['urinecollected']=='1')
]
df_stool=df_stool[['participant_id','urinecollected']]
df_urine=df_sample_collection[
 (df_sample_collection['mstool_time']=='1')
]
df_urine=df_urine[['participant_id','mstool_time']]

df_blood=df_sample_collection[
 (df_sample_collection['matbld_visitdate_q1_e327e1']!='')
]
df_blood=df_blood[['participant_id','matbld_visitdate_q1_e327e1']]

df_stool = df_stool.drop_duplicates()
df_urine = df_urine.drop_duplicates()
df_blood = df_blood.drop_duplicates()

df_merged = pd.merge(df_stool, df_urine, on='participant_id', how='outer')
df_merged = pd.merge(df_merged, df_blood, on='participant_id', how='outer').drop_duplicates('participant_id')





In [59]:
df_collection=df.export_records(forms=['cordcollection'])
df_collection=pd.DataFrame(df_collection)

In [60]:
df_collection=df_collection[
     (df_collection['placenta_available']=='1')
]
df_collection=df_collection[['participant_id','cod_vailable','placenta_available']]
df_collection.to_csv('cord_collection.csv',index=False)

In [61]:
df.forms

['seasonal_blood_collection_2',
 'cordcollection',
 'lmp',
 'obstetric_medicalhistory',
 'socioeconomic_demographic',
 'contraception_medical_smoking_history',
 'dietary',
 'sample_reception_processing',
 'preganancy_urine_collection',
 'preganancy_stool_collection',
 'infant_anthrops',
 'cord_placenta_processing',
 'ultrasound_scan',
 'infant_anthropometry',
 'delivery',
 'recruitment',
 'maternal_health_check',
 'maternal_blood_sample_collection',
 'consent',
 'pregnancy_test_result',
 'prepregnancy_exposure',
 'anthrops',
 'seasonal_blood_reception_rocessing',
 'withdrawal',
 'baby_check']

In [62]:
code_processing=df.export_records(forms=['cord_placenta_processing'])
code_processing=pd.DataFrame(code_processing)

In [63]:
code_processing=code_processing[
    (code_processing['cpp_visit_date']!='')
]

In [64]:
ebrima_requested=pd.read_excel('Ebrima_request.xlsx')

df_merged=pd.merge(ebrima_requested,df_merged,on='participant_id',how='inner')
df_merged.to_csv('Ebrima_requested_update.csv',index=False)

In [65]:
join=pd.merge(del_not_withdrawl,df_ultra,on='participant_id',how='inner')

NameError: name 'del_not_withdrawl' is not defined

In [ ]:
HCG_positive=df.export_records(forms=['pregnancy_test_result'])

In [ ]:
HCG_positive=pd.DataFrame(HCG_positive)

HCG_positive=HCG_positive[
(HCG_positive['pt_testdate_q1']>='2023-12-19')
]
HCG_positive=HCG_positive[
    (HCG_positive['pt_pregnancy_positive_q3']=='1')
].drop_duplicates('participant_id')


